# AA PAN-TRAIN 2018

In [1]:
%matplotlib inline

In [2]:
#python basic libs
import os;
from os.path import join as pathjoin;
import re;
import random;
import json;
from collections import defaultdict;
from pprint import pprint
from time import time

In [3]:
#data analysis libs
import numpy as np;
import pandas as pd;
import seaborn as sns;
import matplotlib.pyplot as plt;

In [4]:
#machine learning libs
#feature extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import feature_selection;

#preprocessing and transformation
from sklearn import preprocessing;
from sklearn.preprocessing import MaxAbsScaler;
from sklearn.decomposition import PCA;

from sklearn.base import BaseEstimator, ClassifierMixin

#classifiers
from sklearn.linear_model import LogisticRegression

#
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

#model valuation
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score;

NLP libraries

In [5]:
!pip install spacy=='2.3' gensim

     |████████████████████████████████| 10.0MB 2.8MB/s 
     |████████████████████████████████| 2.1MB 46.5MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [6]:
!python -m spacy download pt  
!python -m spacy download es
!python -m spacy download it
!python -m spacy download en
!python -m spacy download fr

     |████████████████████████████████| 19.6MB 1.1MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.3.0-cp36-none-any.whl size=19557499 sha256=7a84e683264f0ef33d6afa372641de61302af0022598c2f31e0821376c47416c
  Stored in directory: /tmp/pip-ephem-wheel-cache-d8i2u_1j/wheels/f6/43/f6/949b5e2b48df37aa3ed81fc3f4af4154afa276518cc004b990
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')
     |████████████████████████████████| 16.2MB 649kB/s 
  Created wheel for es-core-news-sm: filename=es_core_news_sm-2.3.1-cp36-none-any.whl size=16217000 sha256=fc9f303b404d9e56984ec9070c6e3a231a95a65c629cfbe5590da4aa6a2e5d69
  Stored in directory: /tmp/pip-ephem-wheel-cache-ihpbrhhb/wheels/75/be/34/88bac54b4d3f462835c18028131

In [7]:
import spacy;
import gensim
import logging
from gensim.models import KeyedVectors
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.CRITICAL)

In [8]:
from IPython.display import Markdown, display, HTML

libraries configurations

In [9]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
warnings.simplefilter(action='ignore', category=UndefinedMetricWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [10]:
np.set_printoptions(precision=4)
pd.options.display.float_format = '{:,.4f}'.format
sns.set(color_codes=True);

In [11]:
import platform;
import sklearn;
import scipy;

Markdown(
    "|%-15s|%-65s|\n"%("PACK","VERSION") +
    "|%-15s|%-65s|\n"%('-'*15,'-'*65) +
    ('\n'.join(
    "|%-15s|%-45s| " % (pack, version.strip())
    for pack, version in
    zip(['SO','NumPy','SciPy','Scikit-Learn','Seaborn','Spacy','Gensim'],
        [platform.platform(), np.__version__, scipy.__version__,
         sklearn.__version__, spacy.__version__, gensim.__version__])

)))

|PACK           |VERSION                                                          |
|---------------|-----------------------------------------------------------------|
|SO             |Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic| 
|NumPy          |1.18.5                                       | 
|SciPy          |1.4.1                                        | 
|Scikit-Learn   |0.22.2.post1                                 | 
|Seaborn        |2.3.0                                        | 
|Spacy          |3.6.0                                        | 

### on google

In [12]:
from google.colab import drive, files
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [13]:
currentCorpus = 'pan18_train';

In [14]:
baseDir = '/content/gdrive/My Drive/Colab Notebooks/AA'
outputDir= pathjoin(baseDir,'ensemble','output_ensemble');

os.makedirs(outputDir, exist_ok=True)

def transferCode(filename):
    with open(os.path.join(baseDir,'ensemble',filename)) as fin:
        with open(filename,'w') as fout:
            fout.write(fin.read());
transferCode('pan.py')
transferCode('sklearnExtensions.py')

In [15]:
import pan
import sklearnExtensions
from sklearnExtensions import DenseTransformer, POSTagTransformer,FilterTagTransformer;

## Data Ingestion

In [16]:
datasets = pd.read_json(baseDir+'/data/AllDS.json.zip', orient='records', compression='gzip')
datasets['row_index'] = np.arange(len(datasets));
datasets = datasets[['row_index','dataset','problem','language','set','filename','text','label']]
print(datasets['dataset'].unique())

['pan18_train' 'pan18_eval' 'lyrics' 'socialaa']


In [17]:
df = datasets.query(f'dataset == "{currentCorpus}"').copy()

# Caching embeddings and POS Tagging

### Linguistic anotations

In [18]:
postaggers = {
    l:POSTagTransformer(language=m)
    for l,m in {'en':'en', 'fr':'fr', 'it':'it', 'sp':'es'}.items()
}

the POSTagTransformer returns a sequence of tokens and its annotations
1. Text
2. tag
3. pos
4. dep

In [19]:
def cachePOS(row):
    if row['language'] == 'pl':
        return [];
    else:
        return postaggers[row['language']].fit_transform([row['text']])[0]

df['POS']=df.apply(cachePOS,axis=1)

In [20]:
df

,row_index,dataset,problem,language,set,filename,text,label,POS
0,0,pan18_train,problem00001,en,known,known00001.txt,"graceful ones.\n\n""One more,"" Marvelous said, ...",candidate00001,"[(graceful, JJ, ADJ, amod), (ones, NNS, NOUN, ..."
1,1,pan18_train,problem00001,en,known,known00002.txt,"before. If he can, he’ll remember a classmate ...",candidate00001,"[(before, RB, ADV, ROOT), (., ., PUNCT, punct)..."
2,2,pan18_train,problem00001,en,known,known00003.txt,she thought - he was in Team Baron only becaus...,candidate00001,"[(she, PRP, PRON, nsubj), (thought, VBD, VERB,..."
3,3,pan18_train,problem00001,en,known,known00007.txt,"As far as she remembers, she's always hated pr...",candidate00001,"[(As, RB, ADV, advmod), (far, RB, ADV, advmod)..."
4,4,pan18_train,problem00001,en,known,known00006.txt,"“Wait for me, please!”\n\nShe glanced towards ...",candidate00001,"[(“, ``, PUNCT, punct), (Wait, VB, VERB, ROOT)..."
...,...,...,...,...,...,...,...,...,...
1491,1491,pan18_train,problem00010,sp,unknown,unknown00060.txt,Tenía el cabello empapado y los labios apretad...,candidate00001,"[(Tenía, VERB__Mood=Ind|Number=Sing|Person=3|T..."
1492,1492,pan18_train,problem00010,sp,unknown,unknown00061.txt,"cargo, el Ministro Kingsley confiaba mucho en ...",candidate00003,"[(cargo, NOUN__Gender=Masc|Number=Sing, NOUN, ..."
1493,1493,pan18_train,problem00010,sp,unknown,unknown00062.txt,manera iba a ser reconocido en Hogwarts. Albus...,candidate00004,"[(manera, NOUN__Gender=Fem|Number=Sing, NOUN, ..."
1494,1494,pan18_train,problem00010,sp,unknown,unknown00063.txt,Tenebroso.\n\n—¿Y tú qué sabes? ¿Eso te lo ens...,candidate00003,"[(Tenebroso, ADJ__Gender=Masc|Number=Sing, ADJ..."


### Embeddings

In [21]:
def loadEmbeddings(lang):
    if lang == 'sp':
        lang = 'es';
    model = KeyedVectors.load_word2vec_format(os.path.join(baseDir,'data','embedding_cache','w2v_'+lang+'.txt.gz'), binary=False, unicode_errors='replace')
    return model;

In [22]:
def problemDoc(df,problem, column='text'):
    df = df.query(f"problem == '{problem}'")

    train = df[df['set'] =='known']
    X_train = np.array([c for c in train[column]])
    y_train = train['label'].values
    index_train = train['row_index'].values

    # The test set is actually the validation set and should not be used in the training processing
    test = df[df['set'] =='unknown']
    X_test = np.array([c for c in test[column]])
    y_test = test['label'].values
    index_test = test['row_index'].values

    return X_train, y_train, index_train, X_test, y_test, index_test;

It was used embeddings processed by Nordic Language Processing Laboratry 
and availabe on http://vectors.nlpl.eu/repository/. This website was visted in  jan 01 2019

In [23]:
def fitPipeline(df, problem,language, pipe,params,caching):
    print ("Problem: %s,  language: %s, " %(problem,language), end=' ');
    
    column = 'text';
    if caching == 'POS':
        column = 'POS';
    
    train_docs, train_labels, index_train, test_docs, test_labels,index_test = problemDoc(df, problem,column);
    if caching == 'POS':
        if language == 'pl':
            print("POS tagger not found")
            return {
                'problem':problem,
                'language':language,
                'n_authors': 0,
                'f1'       : 0,
                'precision': 0,
                'recall'   : 0,
                'accuracy' : 0,
            }, [[],[],[],[],[],[]], {};
    elif caching == 'w2v':
        global embeddingsCacheLanguage;
        global embeddingsCache;
        #yes it is uggly to use global var ... but it saved a lot of memory
        if embeddingsCacheLanguage != language:
            embeddingsCache = loadEmbeddings(language);
            embeddingsCacheLanguage = language;
        
    
    #using language specific params
    if language in params:
        pipe.set_params(**params[language])
        
    t0 = time()
    pipe.fit(train_docs, train_labels)
    print("Fitted in %0.3fs" % (time() - t0), end=' ');
    
    t0 = time()
    train_pred=pipe.predict(train_docs);
    train_prob=pipe.predict_proba(train_docs);

    test_pred=pipe.predict(test_docs);
    test_prob=pipe.predict_proba(test_docs);
    print("Predicted in %0.3fs" % (time() - t0));
    
    f1,precision,recall,accuracy = pan.eval_measures2019(test_labels,test_pred)
    
    return {
        'problem':problem,
        'language':language,
        'n_authors':len(set(train_labels)),
        'f1'       : round(f1,3),
        'precision': round(precision,3),
        'recall'   : round(recall,3),
        'accuracy' : round(accuracy,3),
    }, [train_pred, train_prob,index_train, test_pred,  test_prob,index_test], pipe.get_params();

# Model 1: Character

In [24]:
pipelineChar = Pipeline([
    ('vect',   TfidfVectorizer(analyzer='char',
                               min_df=0.01,
                               max_df=1.0,
                               norm='l2',
                               ngram_range=(2, 5),
                               sublinear_tf=True,
                               lowercase =False)),
    ('scaler', preprocessing.MaxAbsScaler()),
    ('dense',  DenseTransformer()),
    ('transf', PCA(0.999)),
    ('clf', LogisticRegression(random_state=0,multi_class='multinomial', solver='newton-cg')),
])

In [25]:
pipelineCharParams= {
    'it':{
        'vect__ngram_range': (3, 5),
    },
    'pl':{
        'vect__ngram_range': (2, 3),
    }
}

# Model 2: Word

In [26]:
pipelineWord = Pipeline([
    ('vect',   TfidfVectorizer(analyzer='word',
                               norm='l1',
                               min_df=2,
                               max_df=1.0,
                               ngram_range=(1, 2),
                               smooth_idf=True,
                               lowercase =True,
                               sublinear_tf=True)),
    ('dense',  DenseTransformer()),
    ('scaler', preprocessing.MaxAbsScaler()),
    ('transf', PCA(0.999)),
    ('clf', LogisticRegression(random_state=0,multi_class='multinomial', solver='newton-cg')),
])

In [27]:
pipelineWordParams= {
    'en':{
        'vect__ngram_range': (1, 3),
        'vect__norm': 'l2',
    },

    'pl':{
        'vect__norm': 'l2',
     },
}

# Model 3: distortions

In [28]:
distortions = [
    (r"[A-Z]"          ,'U','Upcase '),
    (r"[a-z]"          ,'l','lowercase'),
    (r"[À-ÿĄ-ż]"       ,'d','Diacrit'),
    (r"\d"             ,'D','Digits'),
    (r"[\n\r\t]"       ,'E','Paragraph control'),
    (r"\–|\-|\_|\—"    ,'H','Hyphen'),
    (r"\"|\'|’|[“”]"   ,'Q','quotation'),
    (r"\s"             ,'S','Spaces'),
    (r"\(|\)|\[|\]\{|\}|\«|\»",'G','Groups'),
    (r"…",'','...'),
    (r"\.|\,|[?]|\!|\:|\;|\„|\*|\/"   ,'P','Punctuation'),
    (r"[^aUldDEHQSGP]", 'X','Unkown'),
]

distortions = [(re.compile(d[0]), d[1],d[2]) for d in distortions]

In [29]:
class DistortionTransformer(BaseEstimator):
    def __init__(self,verbose=False):
        self.verbose = verbose;
        
    def _distortion(self,doc):
        for d in distortions:
            doc = d[0].sub(d[1],doc);
        return doc;

    def transform(self, X, y=None):
        t0 = time()
        if self.verbose:
            print('filtered in', end=' ')

        X = [self._distortion(doc) for doc in X]
        
        if self.verbose:
            print(" %0.3fs" % (time() - t0))
        return np.array(X);       

    def fit(self, X, y=None):
        self.is_fitted = True
        return self

    def fit_transform(self, X, y=None):
        return self.transform(X=X, y=y)

In [30]:
pipelineDistortion = Pipeline([
    ('filter',DistortionTransformer()),
    ('vect', TfidfVectorizer(
            analyzer='char',
            min_df=0.01,
            lowercase=False,
            norm='l2',
            sublinear_tf=True,
            ngram_range=(1, 5),
            )
    ),
    ('scaler', MaxAbsScaler()),
    ('dense',  DenseTransformer()),
    ('transf', PCA(0.999)),
    ('clf', LogisticRegression(random_state=0,C=1,multi_class='multinomial', solver='newton-cg')),
])

In [31]:
pipelineDistortionsParams = {
    'en':{
        'clf__C': 0.1,
        'vect__norm': 'l1',        
    },
    'fr':{
        'vect__norm': 'l1',
        'vect__ngram_range': (2, 5),
    },
    'it':{
        'clf__C': 0.1,
    },
    'sp':{
        'vect__norm': 'l1',
    }
}

# Model 4: POS

In [32]:
#train_docs, train_labels, test_docs, test_filename = cachingPOSTAG(problem)

pipelinePOS = Pipeline([
    ('filter',FilterTagTransformer(token='POS')),
    ('vect', TfidfVectorizer(
                        min_df=0.01,
                        lowercase=False,
                        use_idf=True,
                        norm='l1',
                        sublinear_tf=False,
                        ngram_range=(1, 5)
            )),
    ('scaler', MaxAbsScaler()),
    ('dense',  DenseTransformer()),
    ('transf', PCA(0.999)),
    ('clf', LogisticRegression(random_state=0,multi_class='multinomial', solver='newton-cg')),
])

O parametro sublinear_tf=True apresentou performance ruim em todos os casos de teste exceto o melhor.
Como o melhor caso não é garantido nos conjuntos de avaliação, foi decidido que ele seria mantido como False.

O parametro L1 apresentou performance melhor nos conjuntos com 5 candidatos. No entendo, o conjunto de validação conterá de 5 a 20 e portanto foi escolhido manter o valor l2

# Model 5: TAG  (granular POS)

In [33]:
def spaceTokenizer(x):
    return x.split(" ");

In [34]:
pipelineTag = Pipeline([
    ('filter',FilterTagTransformer(token='TAG')),
    ('vect', TfidfVectorizer(
                               tokenizer=spaceTokenizer,
                               min_df=0.01,
                               lowercase=False,
                               sublinear_tf=True,
                               use_idf=True,
                               norm='l2',
                                ngram_range=(1,3),
                              )),
    ('scaler', MaxAbsScaler()),
    ('dense',  DenseTransformer()),
    ('transf', PCA(0.999)),
    ('clf', LogisticRegression(random_state=0,C=0.1,multi_class='multinomial', solver='newton-cg')),
])

In [35]:
pipelineTagParams = {
    'fr':{
        'vect__ngram_range' :(1,5),
    },
    'sp':{
        'vect__ngram_range' :(1,5),
        'clf__C':1,
    }
}

# Model 6: Sintatic Dependency

In [36]:
pipelineDEP = Pipeline([
    ('filter',FilterTagTransformer(token='DEP')),
    ('vect', TfidfVectorizer(
                    analyzer='word',
                    min_df=0.01,
                    lowercase=False,
                    use_idf=True,
                    norm='l2',
                    ngram_range=(1,5),
            )),
    ('scaler', MaxAbsScaler()),
    ('dense',  DenseTransformer()),
    ('transf', PCA(0.999)),
    ('clf', LogisticRegression(random_state=0,C=1,multi_class='multinomial', solver='newton-cg')),
])

In [37]:
pipelineDEPParams = {
    'en':{
        'clf__C':0.1,
    },
    'fr':{
        'clf__C':0.1,
    },
    'it':{
        'vect__ngram_range' :(1,3),
    },
}

# Model 7: Word Embeddings

In [38]:
embeddingsCacheLanguage = None;
embeddingsCache = None;

In [39]:
from scipy import sparse
from sklearn.utils import assert_all_finite

class EmbeddingVectorizer(BaseEstimator):
    def __init__(self,norm=None,analyzer='word',min_df=1,lowercase=False,sublinear_tf=True,use_idf=True,binary=False):
        self.norm        = norm;
        self.analyzer    = analyzer;
        self.min_df      = min_df;
        self.lowercase   = lowercase;
        self.sublinear_tf= sublinear_tf;
        self.use_idf     = use_idf;
        self.binary      = binary;
        
    def fit(self, X, y=None):
        self.dim_ = len(embeddingsCache.vectors[0]);
        #self.dim_ = len(embeddingsCache[list(embeddingsCache.keys())[0]])
        self.vect = TfidfVectorizer(**self.get_params())
        self.vect.fit(X,y);
        self.word_vectors_ = np.zeros((len(self.vect.vocabulary_), self.dim_));
        
        #this arrays says if a word in the vocab has 
        self.missingWord_  = np.zeros(len(self.vect.vocabulary_),dtype=np.int8);
        
        blank = np.zeros(self.dim_);

        for  word, index in self.vect.vocabulary_.items():
            temp = blank.copy();
            for w in [word, word.lower(), word.capitalize()]:
                if w in embeddingsCache:
                    temp = embeddingsCache[w];
                    break;
            if temp.sum() == 0 :
                self.missingWord_[index] = 1;
            self.word_vectors_[index,:] = temp;
        return self
    
    def transform(self, X):
        import gc;
        gc.collect()
        X = self.vect.transform(X);
        
        X = sparse.csc_matrix(X);
        V = sparse.csc_matrix(self.word_vectors_);
        M = sparse.csc_matrix(self.missingWord_).T;
        
        missing = preprocessing.normalize((X>0).dot(M),norm='l1');
        missing = sparse.csc_matrix(missing.reshape(-1,1));
        
        #memory friendly dot product
        X  = preprocessing.normalize(X.dot(V),norm='l1');
        
        #last columns will contain the embedding coverage
        X = sparse.hstack([X,missing]);
        
        return X;
    
    def fit_transform(self, X, y=None):
        self.fit(X,y);
        return self.transform(X)
    
    def get_feature_names():
        return self.vect.get_feature_names();

In [40]:
pipelineEmbeddings = Pipeline([
    ('vect', EmbeddingVectorizer(
            analyzer='word',
            min_df=0.01,
            lowercase=False,
            sublinear_tf=False,
            norm='l1',
            use_idf=False
            )
    ),
    ('scaler',preprocessing.MaxAbsScaler()),
    ('dense',DenseTransformer()),
    ('pca',PCA(0.999)),
    ('clf', LogisticRegression(random_state=0,C=1,multi_class='multinomial', solver='newton-cg')),
])

In [41]:
pipelineEmbeddingsParams = {
    'fr':{
        'vect__sublinear_tf': True,
        'vect__use_idf': True,
    },

    'it':{
        'vect__use_idf': True,
    },

    'pl':{
        'clf__C': 0.1,
        'vect__use_idf': False,
        'vect__sublinear_tf': True,
    },
    'sp':{
        'vect__sublinear_tf': True,
        'vect__use_idf': True,
    },
}

# Applying models

In [42]:
models =[
    ('char',pipelineChar,pipelineCharParams,None),
    ('word',pipelineWord,pipelineWordParams,None),
    ('dist',pipelineDistortion,pipelineDistortionsParams,None),
    ('pos',pipelinePOS,{},'POS'),
    ('tag',pipelineTag,pipelineTagParams,'POS'),
    ('dep',pipelineDEP,pipelineDEPParams,'POS'),
    ('w2v',pipelineEmbeddings,pipelineEmbeddingsParams,'w2v'),
]

results = {};

for model in models:
    if model[0] in results:
        continue;
    print(('_'*100)+'\n'+model[0]+'\n'+('_'*100))
    result= {
        'metric':[],
        'pred_train':[],
        'pred_test':[],
        'prob_train':[],
        'prob_test':[],
        'index_train':[],
        'index_test':[],
        'params':[],
    }
    for problem in np.sort(df['problem'].unique()):
        language = df.query(f'problem =="{problem}"')['language'].unique()[0]
        metric, vects, params = fitPipeline(df, problem, language, model[1], model[2], caching=model[3]);
        result['metric'].append(metric);

        result['pred_train'].append(vects[0]);
        result['prob_train'].append(vects[1]);
        result['index_train'].append(vects[2]);

        result['pred_test'].append(vects[3]);
        result['prob_test'].append(vects[4]);
        result['index_test'].append(vects[5]);
        result['params'].append(params);
    results[model[0]]=result;

____________________________________________________________________________________________________
char
____________________________________________________________________________________________________
Problem: problem00001,  language: en,  Fitted in 6.018s Predicted in 6.995s
Problem: problem00002,  language: en,  Fitted in 0.981s Predicted in 1.528s
Problem: problem00003,  language: fr,  Fitted in 5.782s Predicted in 5.571s
Problem: problem00004,  language: fr,  Fitted in 1.021s Predicted in 1.663s
Problem: problem00005,  language: it,  Fitted in 5.556s Predicted in 5.638s
Problem: problem00006,  language: it,  Fitted in 0.978s Predicted in 2.047s
Problem: problem00007,  language: pl,  Fitted in 1.139s Predicted in 3.062s
Problem: problem00008,  language: pl,  Fitted in 0.294s Predicted in 0.731s
Problem: problem00009,  language: sp,  Fitted in 0.904s Predicted in 2.822s
Problem: problem00010,  language: sp,  Fitted in 0.270s Predicted in 1.202s
_________________________________

In [43]:
dfMetric = pd.DataFrame();
for m in results:
    df_temp = pd.DataFrame(results[m]['metric'])[['problem','n_authors','language','f1','precision','recall','accuracy']]
    df_temp['model']=m
    dfMetric= dfMetric.append(df_temp)

with pd.option_context("display.precision", 3):
    display(dfMetric.fillna(value=0) \
        .pivot(index='problem',columns='model',values=['f1','accuracy']).round(3)\
        .style.background_gradient(subset=['f1'], axis=1, cmap=plt.cm.Blues)\
        .background_gradient(subset=['accuracy'], axis=1, cmap=plt.cm.Greens))

# Dynamic Ensemble

In [44]:
def buildClassifier():
    return Pipeline(steps=[
                ('pca',PCA(0.9999)),
                ('clf', LogisticRegression(random_state=0,C=1,max_iter=3000))
            ])

In [45]:
def concatenatXVectors(results, problemIndex, models):
    x_train = [results[model]['prob_train'][problemIndex] for model in models];
    x_test  = [results[model]['prob_test'][problemIndex] for model in models];
    
    x_train = [x for x in x_train if len(x) > 1];
    x_test  = [x for x in x_test  if len(x) > 1];
    
    if len(x_train) < 1:
        return None,None;
    
    return np.hstack(x_train),  np.hstack(x_test);

In [46]:
def sortModels(results, problemIndex,models,train_labels, test_labels):
    f1 = [];
    
    for model in models:
        x_train, x_test = concatenatXVectors(results, problemIndex, [model]);
        if x_train is None:
            f1.append(0);
        else:
            pipe = buildClassifier()
            pred_label = pipe.fit(x_train, train_labels).predict(x_test);
            f1.append(f1_score(test_labels, pred_label,average='macro'));
    models = np.array(models);
    f1 = np.array(f1);
    
    models = models[f1 >0];
    f1 = f1[f1 >0]
    
    return list(models[np.argsort(-f1)]);

In [47]:
resultFinal = []
bestModels = []
predictions = [];
for problemIndex, problem in enumerate(df['problem'].unique()):
    print('\n'+problem)

    train_docs, train_labels, index_train, test_docs, test_labels,index_test = problemDoc(df, problem);

    #forward step: sorting the models by
    models = sortModels(results, problemIndex,list(results.keys()), train_labels, test_labels);

    print("sorted "+str(models))

    #step 0 - train only the best model.
    bestModel =[ models.pop(0)];
    x_train, x_test = concatenatXVectors(results, problemIndex, bestModel);
    pred_label  = buildClassifier().fit(x_train, train_labels).predict(x_test);
    bestF1      = f1_score(test_labels,pred_label,average='macro')
    models = list(set(models)-set(bestModel)); # removing the current model from the list.
    
    improve = True;
    iteration = 0;
    while (improve):
        improve = False;
        iteration +=1 ;
        print("Iteration "+str(iteration)    +" Best model:     %-77s %.4f"%(str(bestModel),bestF1))
        for model in models:
            currentModel = bestModel +[model];
            print("Iteration "+str(iteration)+" current model:  %-75s "%str(currentModel), end=' ')

            x_train, x_test = concatenatXVectors(results, problemIndex, currentModel);

            pipe = buildClassifier()

            pred_label = pipe.fit(x_train, train_labels).predict(x_test);
            currentF1 = f1_score(test_labels,pred_label,average='macro');
            
            print(" %.4f"%currentF1)


            if currentF1/bestF1 > 1.015:
                improve = True;
                bestF1 = currentF1;
                bestModel = currentModel;
        models = list(set(models)-set(currentModel));
    print("Selected Model: "+str(bestModel))
    bestModels.append(bestModel);
    
    x_train, x_test = concatenatXVectors(results, problemIndex, bestModel);
    
    pipelineFinal = Pipeline(steps=[
        ('pca',PCA(0.9999)),
        ('clf', LogisticRegression(random_state=0,C=100000))
    ])

    pipelineFinal.fit(x_train, train_labels);
    pred_train = pipelineFinal.predict(x_train);
    pred_proba_train = pipelineFinal.predict_proba(x_train);
    pred_test = pipelineFinal.predict(x_test);
    pred_proba_test = pipelineFinal.predict_proba(x_test);

    def appendPrediction(predictions,classes_,index, pred,proba):
        for i,p, pr in  zip(index, pred, proba):
            predictions.append(dict(**{
                    'dataset':currentCorpus,
                    'problem':problem,
                    'model':'dymAA',
                    'row_index':i,
                    'pred':p,
                }, **{
                    cc:pr_ for cc,pr_ in zip(classes_, pr)
                })
            )

    appendPrediction(predictions,pipelineFinal.classes_, index_train, pred_train, pred_proba_train)
    appendPrediction(predictions,pipelineFinal.classes_, index_test, pred_test, pred_proba_test)
    

    resultFinal.append({
        'model':'0ensemble',
        'problem':problem,
        'language':df.query(f"problem == '{problem}'")['language'].unique()[0],
        'n_authors': len(set(train_labels)),
        'f1':round(f1_score(test_labels,pred_test,average='macro'),3),
        'accuracy':round(accuracy_score(test_labels,pred_test),3)
    })


problem00001
sorted ['char', 'word', 'tag', 'dist', 'dep', 'pos', 'w2v']
Iteration 1 Best model:     ['char']                                                                      0.6533
Iteration 1 current model:  ['char', 'pos']                                                               0.5550
Iteration 1 current model:  ['char', 'dep']                                                               0.5819
Iteration 1 current model:  ['char', 'dist']                                                              0.5946
Iteration 1 current model:  ['char', 'word']                                                              0.6569
Iteration 1 current model:  ['char', 'tag']                                                               0.5958
Iteration 1 current model:  ['char', 'w2v']                                                               0.4512
Selected Model: ['char']

problem00002
sorted ['pos', 'word', 'char', 'tag', 'dep', 'w2v', 'dist']
Iteration 1 Best model:     ['pos'] 

In [48]:
pd.concat([dfMetric, pd.DataFrame(resultFinal)]).fillna(value=0) \
    .pivot(index='problem',columns='model',values=['f1','accuracy']).fillna(0)\
    .style.background_gradient(subset=['f1'], axis=1, cmap=plt.cm.Blues)\
    .background_gradient(subset=['accuracy'], axis=1, cmap=plt.cm.Greens)\
    .format('{:,.2f}'.format)

### Latex version

In [49]:
print(pd.concat([dfMetric, pd.DataFrame(resultFinal)]).fillna(value=0) \
    .pivot(index='problem',columns='model',values='f1').reset_index()\
    .fillna(0).to_latex(index=False, float_format='{:,.2f}'.format))

\begin{tabular}{lrrrrrrrr}
\toprule
      problem &  0ensemble &  char &  dep &  dist &  pos &  tag &  w2v &  word \\
\midrule
 problem00001 &       0.67 &  0.65 & 0.39 &  0.39 & 0.32 & 0.46 & 0.27 &  0.49 \\
 problem00002 &       0.67 &  0.55 & 0.50 &  0.33 & 0.59 & 0.53 & 0.31 &  0.58 \\
 problem00003 &       0.76 &  0.68 & 0.45 &  0.55 & 0.40 & 0.44 & 0.23 &  0.51 \\
 problem00004 &       0.87 &  0.79 & 0.65 &  0.75 & 0.77 & 0.77 & 0.21 &  0.57 \\
 problem00005 &       0.66 &  0.60 & 0.28 &  0.45 & 0.26 & 0.58 & 0.26 &  0.56 \\
 problem00006 &       0.79 &  0.63 & 0.56 &  0.44 & 0.33 & 0.57 & 0.30 &  0.52 \\
 problem00007 &       0.52 &  0.43 & 0.00 &  0.29 & 0.00 & 0.00 & 0.07 &  0.43 \\
 problem00008 &       0.92 &  0.74 & 0.00 &  0.82 & 0.00 & 0.00 & 0.50 &  0.92 \\
 problem00009 &       0.81 &  0.80 & 0.34 &  0.46 & 0.38 & 0.46 & 0.19 &  0.69 \\
 problem00010 &       0.86 &  0.86 & 0.59 &  0.63 & 0.59 & 0.66 & 0.39 &  0.59 \\
\bottomrule
\end{tabular}



In [50]:
# mean by model
print(pd.concat([dfMetric, pd.DataFrame(resultFinal)]).fillna(value=0) \
    .pivot(index='problem',columns='model',values='f1').reset_index().mean().to_frame().T\
     .to_latex(index=False, float_format='{:,.2f}'.format))

\begin{tabular}{rrrrrrrr}
\toprule
 0ensemble &  char &  dep &  dist &  pos &  tag &  w2v &  word \\
\midrule
      0.75 &  0.67 & 0.38 &  0.51 & 0.36 & 0.45 & 0.27 &  0.59 \\
\bottomrule
\end{tabular}



In [51]:
for problemIndex, problem in enumerate(df['problem'].unique()):
    lang = df.query(f"problem == '{problem}'")['language'].unique()[0]
    print(problem +' '+lang+' '+ str(bestModels[problemIndex]))

problem00001 en ['char']
problem00002 en ['pos', 'word', 'w2v', 'tag', 'char']
problem00003 fr ['char', 'pos', 'dep']
problem00004 fr ['char', 'pos']
problem00005 it ['char', 'dep', 'word', 'tag']
problem00006 it ['char', 'dep', 'tag', 'word']
problem00007 pl ['char', 'dist', 'word', 'w2v']
problem00008 pl ['word']
problem00009 sp ['char']
problem00010 sp ['char']


In [52]:
pd.pivot_table(
    pd.concat([dfMetric, pd.DataFrame(resultFinal)]),
    index='model',
    columns='n_authors',
    values='f1',
    aggfunc='mean',
    fill_value=0).T

model,0ensemble,char,dep,dist,pos,tag,w2v,word
n_authors,,,,,,,,
0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.8206,0.7146,0.5755,0.5954,0.5705,0.6330,0.3412,0.6358
20,0.6838,0.6308,0.3638,0.4262,0.3410,0.4843,0.2016,0.5356


In [53]:
pd.pivot_table(
    pd.concat([dfMetric, pd.DataFrame(resultFinal)]),
    index='model',
    columns='language',
    values='f1',
    aggfunc='mean',
    fill_value=0)

language,en,fr,it,pl,sp
model,,,,,
0ensemble,0.6700,0.8145,0.7235,0.7185,0.8345
char,0.6010,0.7335,0.6170,0.5830,0.8290
dep,0.4445,0.5475,0.4190,0.0000,0.4675
dist,0.3595,0.6505,0.4445,0.5530,0.5465
pos,0.4590,0.5830,0.2950,0.0000,0.4860
tag,0.4935,0.6070,0.5730,0.0000,0.5610
w2v,0.2885,0.2170,0.2805,0.2825,0.2885
word,0.5330,0.5385,0.5415,0.6745,0.6410


In [54]:
pd.pivot_table(
    pd.concat([dfMetric, pd.DataFrame(resultFinal)]),
    index='model',
    columns=['language','n_authors'],
    values='f1',
    aggfunc='mean',
    fill_value=0)

language      en            fr            it  ... pl                   sp       
n_authors     5      20     5      20     5   ... 0      5      20     5      20
model                                         ...                               
0ensemble 0.6670 0.6730 0.8670 0.7620 0.7850  ...  0 0.9220 0.5150 0.8620 0.8070
char      0.5490 0.6530 0.7920 0.6750 0.6320  ...  0 0.7380 0.4280 0.8620 0.7960
dep       0.5000 0.3890 0.6460 0.4490 0.5610  ...  0 0.0000 0.0000 0.5950 0.3400
dist      0.3320 0.3870 0.7490 0.5520 0.4430  ...  0 0.8200 0.2860 0.6330 0.4600
pos       0.5950 0.3230 0.7710 0.3950 0.3270  ...  0 0.0000 0.0000 0.5890 0.3830
tag       0.5300 0.4570 0.7710 0.4430 0.5670  ...  0 0.0000 0.0000 0.6640 0.4580
w2v       0.3100 0.2670 0.2090 0.2250 0.3010  ...  0 0.4970 0.0680 0.3890 0.1880
word      0.5770 0.4890 0.5720 0.5050 0.5200  ...  0 0.9220 0.4270 0.5880 0.6940

[8 rows x 11 columns]

In [55]:
pd.pivot_table(
    pd.concat([ pd.DataFrame(resultFinal)]),
    index='model',
    columns=['language','n_authors'],
    values='f1',
    aggfunc='mean',
    fill_value=0)

language      en            fr            it            pl            sp       
n_authors     5      20     5      20     5      20     5      20     5      20
model                                                                          
0ensemble 0.6670 0.6730 0.8670 0.7620 0.7850 0.6620 0.9220 0.5150 0.8620 0.8070

In [56]:
pd.DataFrame(predictions).round(5).sort_values('row_index')

,dataset,problem,model,row_index,pred,candidate00001,candidate00002,candidate00003,candidate00004,candidate00005,candidate00006,candidate00007,candidate00008,candidate00009,candidate00010,candidate00011,candidate00012,candidate00013,candidate00014,candidate00015,candidate00016,candidate00017,candidate00018,candidate00019,candidate00020
0,pan18_train,problem00001,dymAA,0,candidate00001,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,pan18_train,problem00001,dymAA,1,candidate00001,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
2,pan18_train,problem00001,dymAA,2,candidate00001,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
3,pan18_train,problem00001,dymAA,3,candidate00001,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
4,pan18_train,problem00001,dymAA,4,candidate00001,1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1491,pan18_train,problem00010,dymAA,1491,candidate00001,0.8011,0.0262,0.0867,0.0251,0.0608,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1492,pan18_train,problem00010,dymAA,1492,candidate00002,0.0427,0.3175,0.2857,0.1672,0.1868,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1493,pan18_train,problem00010,dymAA,1493,candidate00004,0.0784,0.2172,0.2967,0.3520,0.0557,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1494,pan18_train,problem00010,dymAA,1494,candidate00003,0.0186,0.3228,0.3730,0.2406,0.0450,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


## Saving data

In [57]:
import pickle;
with open(pathjoin(outputDir,f'{currentCorpus}_parcial.pkl'),"wb") as f:
    pickle.dump(results,f)


pd.DataFrame(predictions)\
    .rename(columns={'index':'row_index'}).sort_values('row_index').round(5)\
    .to_csv(pathjoin(outputDir,currentCorpus+'_predictions.csv.zip'), index=False, compression='zip', encoding='utf-8')

In [58]:
def resultsToTable(results):
    temp = []
    def unfold(r, var):
        return [i for j in r[var] for i in j]
    for model in results:
        r = results[model];
        t = {
            'row_index': unfold(r,'index_train') +unfold(r,'index_test'),
            'pred': unfold(r,'pred_train') +unfold(r,'pred_test')
            
        }
        t['model'] = [model] * len(t['row_index'])
        t['dataset'] = [currentCorpus] * len(t['row_index'])
        
        temp.append(t)

    
    return pd.concat([pd.DataFrame(d) for d in temp]);
        

resultsToTable(results) \
    .merge(datasets[['row_index','problem']], on='row_index', how='left')\
    .to_csv(pathjoin(outputDir,currentCorpus+'_predictions_parcial.csv.zip'), index=False, compression='zip', encoding='utf-8')